In [1]:
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, Perceptron, SGDRegressor, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
from sklearn.svm import SVC
import scipy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pickle
from joblib import dump, load
from sklearn.neighbors import KNeighborsRegressor

In [2]:
data = pd.read_csv("/home/mike/python_files/ML_services/data/NHANES_age_prediction.csv")
data["age_group"]
# data = pd.read_csv("/home/mike/python_files/ML_services/data/NHANES_age_prediction.csv").drop("age_group", axis=1)
data

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,73564.0,Adult,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91
1,73568.0,Adult,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85
2,73576.0,Adult,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14
3,73577.0,Adult,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15
4,73580.0,Adult,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92
...,...,...,...,...,...,...,...,...,...,...
2273,83711.0,Adult,38.0,2.0,2.0,33.5,100.0,2.0,73.0,6.53
2274,83712.0,Adult,61.0,1.0,2.0,30.0,93.0,2.0,208.0,13.02
2275,83713.0,Adult,34.0,1.0,2.0,23.7,103.0,2.0,124.0,21.41
2276,83718.0,Adult,60.0,2.0,2.0,27.4,90.0,2.0,108.0,4.99


In [3]:
le = LabelEncoder()

In [4]:
le.fit(data["age_group"])
data["age_group"] = le.transform(data["age_group"])
data

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,73564.0,0,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91
1,73568.0,0,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85
2,73576.0,0,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14
3,73577.0,0,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15
4,73580.0,0,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92
...,...,...,...,...,...,...,...,...,...,...
2273,83711.0,0,38.0,2.0,2.0,33.5,100.0,2.0,73.0,6.53
2274,83712.0,0,61.0,1.0,2.0,30.0,93.0,2.0,208.0,13.02
2275,83713.0,0,34.0,1.0,2.0,23.7,103.0,2.0,124.0,21.41
2276,83718.0,0,60.0,2.0,2.0,27.4,90.0,2.0,108.0,4.99


In [5]:
# train_X, test_X, train_y, test_y = train_test_split(data.drop(["age_group", "RIDAGEYR"], axis=1), data["age_group"], test_size=0.2, stratify=data["age_group"])
train_X, test_X, train_y, test_y = train_test_split(data.drop("RIDAGEYR", axis=1), data["RIDAGEYR"], test_size=0.2, stratify=data["RIDAGEYR"])
mms = MinMaxScaler()
mms.fit(pd.concat((train_X,test_X)).sort_index())
train_X = pd.DataFrame(mms.transform(train_X))
test_X = pd.DataFrame(mms.transform(test_X))
train_X

,0,1,2,3,4,5,6,7,8
0,0.912624,0.0,1.0,0.166667,0.082734,0.093567,0.5,0.092199,0.018698
1,0.731674,0.0,0.0,0.166667,0.161871,0.076023,0.5,0.074468,0.062457
2,0.443668,0.0,0.0,0.166667,0.095324,0.108187,0.5,0.093972,0.062849
3,0.384139,0.0,0.0,0.166667,0.266187,0.046784,0.5,0.062057,0.018796
4,0.153400,0.0,0.0,0.166667,0.408273,0.084795,0.5,0.093972,0.196378
...,...,...,...,...,...,...,...,...,...
1817,0.512152,0.0,0.0,0.000000,0.142086,0.084795,0.5,0.047872,0.021831
1818,0.586146,0.0,0.0,0.166667,0.188849,0.187135,0.5,0.453901,0.073226
1819,0.659057,1.0,1.0,0.166667,0.208633,0.061404,0.5,0.065603,0.034068
1820,0.680901,0.0,0.0,0.166667,0.120504,0.093567,0.5,0.086879,0.046892


In [6]:
lr = LinearRegression()
lr.fit(train_X, train_y)
display(mean_squared_error(test_y, lr.predict(test_X)))
display(mean_absolute_error(test_y, lr.predict(test_X)))
lr_pickle = pickle.dumps(lr)
dump(lr_pickle, '/home/mike/python_files/ML_services/ml_models/LinearRegression_pickle.joblib') 

187.0860544960744

11.297840707328941

['/home/mike/python_files/ML_services/ml_models/LinearRegression_pickle.joblib']

In [7]:
params = {"penalty":("l2","l1","elasticnet"), "alpha":[0.01,0.001,0.0001,0.00001]}
grcv = GridSearchCV(estimator=Perceptron(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
best_perceptron_pickle = pickle.dumps(grcv.best_estimator_)
dump(best_perceptron_pickle, '/home/mike/python_files/ML_services/ml_models/best_perceptron_pickle.joblib') 

559.7149122807018

18.583333333333332

['/home/mike/python_files/ML_services/ml_models/best_perceptron_pickle.joblib']

In [8]:
params = {"n_neighbors":list(range(2, 15)), 'weights':('uniform', 'distance'),}
grcv = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
kneighborsregressor_pickle = pickle.dumps(grcv.best_estimator_)
dump(kneighborsregressor_pickle, '/home/mike/python_files/ML_services/ml_models/KNeighborsRegressor.joblib') 

181.13027320070307

10.899070385399481

['/home/mike/python_files/ML_services/ml_models/KNeighborsRegressor.joblib']

In [9]:
params = {"penalty":("l2","l1","elasticnet"), "alpha":[0.01,0.001,0.0001,0.00001]}
grcv = GridSearchCV(estimator=SGDRegressor(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
SGDRegressor_pickle = pickle.dumps(grcv.best_estimator_)
dump(SGDRegressor_pickle, '/home/mike/python_files/ML_services/ml_models/SGDRegressor.joblib') 

193.42130837896622

11.60798757502636

['/home/mike/python_files/ML_services/ml_models/SGDRegressor.joblib']

In [10]:
np.median(abs(test_y.to_numpy()-grcv.predict(test_X)))

10.342903137575647